In [1]:
!pip install langchain langchain-community langchain-groq neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Unin

In [2]:
NEO4J_URI="neo4j+s://67a1c11e.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="GrRpX_Jj-JBPB4mbpRDIMNvdvb2z1GpSAd8Xocu7BFQ"

In [3]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD"),
)

In [5]:
graph


In [6]:
groq_api_key = "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"



In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-IT")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x792a9e25a590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x792a9e25abf0>, model_name='Gemma2-9b-IT', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.documents import Document
text = """
Time Series Models:
Autoregressive (AR) Models: A model that predicts future values based on past values.
Moving Average (MA) Models: A model that predicts future values based on the average of past errors.
ARIMA Models: A model that combines AR and MA models to predict future values.
"""


In [9]:
documents = [Document(page_content=text)]

In [10]:
documents

[Document(metadata={}, page_content='\nTime Series Models:\nAutoregressive (AR) Models: A model that predicts future values based on past values.\nMoving Average (MA) Models: A model that predicts future values based on the average of past errors.\nARIMA Models: A model that combines AR and MA models to predict future values.\n')]

In [11]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 4.0 MB/s eta 0:00:00


In [12]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_graph_transformer = LLMGraphTransformer(
    llm=llm
)

In [13]:
graph_docs = llm_graph_transformer.convert_to_graph_documents(documents)

In [14]:
graph_docs

[GraphDocument(nodes=[Node(id='Autoregressive (Ar) Models', type='Model', properties={}), Node(id='Moving Average (Ma) Models', type='Model', properties={}), Node(id='Arima Models', type='Model', properties={})], relationships=[Relationship(source=Node(id='Autoregressive (Ar) Models', type='Model', properties={}), target=Node(id='Arima Models', type='Model', properties={}), type='IS_PART_OF', properties={}), Relationship(source=Node(id='Moving Average (Ma) Models', type='Model', properties={}), target=Node(id='Arima Models', type='Model', properties={}), type='IS_PART_OF', properties={})], source=Document(metadata={}, page_content='\nTime Series Models:\nAutoregressive (AR) Models: A model that predicts future values based on past values.\nMoving Average (MA) Models: A model that predicts future values based on the average of past errors.\nARIMA Models: A model that combines AR and MA models to predict future values.\n'))]

In [15]:
graph_docs[0].nodes

[Node(id='Autoregressive (Ar) Models', type='Model', properties={}),
 Node(id='Moving Average (Ma) Models', type='Model', properties={}),
 Node(id='Arima Models', type='Model', properties={})]

# Rag pipeline

In [ ]:
financial_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [ ]:
graph

In [ ]:
graph.query(financial_query)
#

[]

In [26]:
disaster_query="""
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/keshavsingh-ks/Insurance-chat-bot/refs/heads/main/1970-2021_DISASTERS.xlsx%20-%20emdat%20data.csv' AS row

// Create or match Country nodes
MERGE (c:Country {iso: row.ISO})
SET c.name = row.Country,
    c.region = row.Region,
    c.continent = row.Continent

// Create Disaster nodes
MERGE (d:Disaster {id: row.`Dis No`})
SET d.year = toInteger(row.Year),
    d.type = row.`Disaster Type`,
    d.subtype = row.`Disaster Subtype`,
    d.subsubtype = row.`Disaster Subsubtype`,
    d.event_name = row.`Event Name`,
    d.group = row.`Disaster Group`,
    d.subgroup = row.`Disaster Subgroup`,
    d.associated_disaster = row.`Associated Dis`,
    d.associated_disaster2 = row.`Associated Dis2`,
    d.ofda_response = row.`OFDA Response`,
    d.appeal = row.Appeal,
    d.declaration = row.Declaration,
    d.aid_contribution = row.`Aid Contribution`,
    d.magnitude_value = toFloat(row.`Dis Mag Value`),
    d.magnitude_scale = row.`Dis Mag Scale`,
    d.latitude = toFloat(row.Latitude),
    d.longitude = toFloat(row.Longitude),
    d.local_time = row.`Local Time`,
    d.river_basin = row.`River Basin`

// Create Disaster Impact nodes
MERGE (i:Impact {id: row.`Dis No`})
SET i.total_deaths = toInteger(row.`Total Deaths`),
    i.no_injured = toInteger(row.`No Injured`),
    i.no_affected = toInteger(row.`No Affected`),
    i.no_homeless = toInteger(row.`No Homeless`),
    i.total_affected = toInteger(row.`Total Affected`),
    i.reconstruction_costs = toFloat(row.`Reconstruction Costs ('000 US$)`),
    i.insured_damages = toFloat(row.`Insured Damages ('000 US$)`),
    i.total_damages = toFloat(row.`Total Damages ('000 US$)`),
    i.cpi = toFloat(row.CPI),
    i.current_ratio = toFloat(row.`Current Ratio`),
    i.debt_equity_ratio = toFloat(row.`Debt/Equity Ratio`),
    i.roe = toFloat(row.ROE),
    i.roa = toFloat(row.ROA),
    i.roi = toFloat(row.ROI),
    i.net_profit_margin = toFloat(row.`Net Profit Margin`),
    i.free_cash_flow_per_share = toFloat(row.`Free Cash Flow per Share`),
    i.return_on_tangible_equity = toFloat(row.`Return on Tangible Equity`),
    i.inflation_rate = toFloat(row.`Inflation Rate(in US)`),
    i.employees = toInteger(row.`Number of Employees`)

// Link Disaster to Country and Impact
MERGE (c)-[:SUFFERED_FROM]->(d)
MERGE (d)-[:HAS_IMPACT]->(i)
"""

In [22]:
graph

In [28]:
graph.query(disaster_query)

[]

In [29]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Country {name: STRING, iso: STRING, region: STRING, continent: STRING}
Disaster {id: STRING, year: INTEGER, type: STRING, group: STRING, subgroup: STRING, magnitude_scale: STRING, subtype: STRING, event_name: STRING, ofda_response: STRING, subsubtype: STRING, appeal: STRING, declaration: STRING, magnitude_value: FLOAT, latitude: FLOAT, longitude: FLOAT, local_time: STRING, associated_disaster: STRING, associated_disaster2: STRING, aid_contribution: STRING, river_basin: STRING}
Impact {id: STRING, total_deaths: INTEGER, cpi: FLOAT, total_damages: FLOAT, no_affected: INTEGER, total_affected: INTEGER, no_injured: INTEGER, no_homeless: INTEGER, insured_damages: FLOAT, reconstruction_costs: FLOAT}
Relationship properties:

The relationships:
(:Country)-[:SUFFERED_FROM]->(:Disaster)
(:Disaster)-[:HAS_IMPACT]->(:Impact)


In [30]:
from langchain.chains import GraphCypherQAChain
chaim = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

In [31]:
chaim

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x792ad8ec2dd0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x792a9e25a590>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x792a9e25abf0>, model_name='Gemma2-9b-I

In [32]:
response = chaim.invoke({"query":"give all subtype and types of disaster"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disaster) RETURN DISTINCT d.subtype, d.type  

Full Context:
[{'d.subtype': 'Tropical cyclone', 'd.type': 'Storm'}, {'d.subtype': None, 'd.type': 'Flood'}, {'d.subtype': None, 'd.type': 'Storm'}, {'d.subtype': 'Drought', 'd.type': 'Drought'}, {'d.subtype': 'Avalanche', 'd.type': 'Landslide'}, {'d.subtype': 'Ground movement', 'd.type': 'Earthquake'}, {'d.subtype': 'Bacterial disease', 'd.type': 'Epidemic'}, {'d.subtype': 'Flash flood', 'd.type': 'Flood'}, {'d.subtype': 'Land fire (Brush, Bush, Pasture)', 'd.type': 'Wildfire'}, {'d.subtype': 'Landslide', 'd.type': 'Landslide'}]

> Finished chain.


{'query': 'give all subtype and types of disaster',
 'result': 'Tropical cyclone, Storm, Flood, Drought, Avalanche, Landslide, Earthquake, Bacterial disease, Epidemic, Flash flood, Wildfire, Landslide  \n'}